In [14]:

from poker_dataset import PokerDataset
from poker_test import get_models_accuracy
from poker_baseline_model import baseline_model
from poker_model import CustomFCNetwork
from helper import ACTION_NAMES

import torch

In [2]:
filepath = r'C:\Users\kctak\Documents\Code\PokerAI\APS360 Progress\poker_data'
poker_dataset = PokerDataset(filepath, "IlxxxlI", balance_data=True)
print({action_name: len(indices) for action_name, indices in poker_dataset.output_indices.items()})

Processing Items: 100%|██████████| 50664/50664 [00:35<00:00, 1440.72it/s]

{'calls': 39175, 'raises': 39175, 'bets': 0, 'folds': 39175, 'checks': 39175, 'allin': 39175}


In [3]:
import torch.utils

test_frac = 0.15
val_frac = 0.15
train_frac = 1 - test_frac - val_frac

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(poker_dataset, [train_frac, val_frac, test_frac])

In [4]:
# Load trained model 
hidden_shape = [256, 128]
trained_model_path = r'Results2\poker_model.pth'

model = CustomFCNetwork(hidden_shape=hidden_shape)
model.load_state_dict(torch.load(trained_model_path))

<All keys matched successfully>

In [5]:
model_tracker, baseline_tracker = get_models_accuracy(model, baseline_model, test_dataset)

In [7]:
# model_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}
# baseline_tracker = {action_name: {'correct': [], 'wrong': []} for action_name in ACTION_NAMES}

model_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in model_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}
baseline_corr_count = {action_name: {keys: len(values) for keys, values in records.items()} for action_name, records in baseline_tracker.items() if not all([len(values)==0 for keys, values in records.items()])}

print("Model Correct Count", model_corr_count)
print("Baseline Correct Count", baseline_corr_count)

Model Correct Count {'calls': {'correct': 5900, 'wrong': 47}, 'raises': {'correct': 5792, 'wrong': 27}, 'folds': {'correct': 5737, 'wrong': 23}, 'checks': {'correct': 5964, 'wrong': 4}, 'allin': {'correct': 5887, 'wrong': 0}}
Baseline Correct Count {'calls': {'correct': 1766, 'wrong': 2766}, 'folds': {'correct': 5612, 'wrong': 17352}, 'allin': {'correct': 974, 'wrong': 911}}


In [9]:
model_accuracy = sum([records['correct'] for records in model_corr_count.values()]) / sum([records['correct'] + records['wrong'] for records in model_corr_count.values()])
baseline_accuracy = sum([records['correct'] for records in baseline_corr_count.values()]) / sum([records['correct'] + records['wrong'] for records in baseline_corr_count.values()])

print("Model Accuracy", model_accuracy)
print("Baseline Accuracy", baseline_accuracy)

Model Accuracy 0.9965624042748715
Baseline Accuracy 0.2842653415472584


In [16]:
sample_action_name = 'calls'
sample_truth = 'correct'

model_io_records = model_tracker[sample_action_name][sample_truth]
baseline_io_records = baseline_tracker[sample_action_name][sample_truth]

# Choose a random int in the results
model_io_record = model_io_records[torch.randint(0, len(model_io_records), (1,)).item()]
baseline_io_record = baseline_io_records[torch.randint(0, len(baseline_io_records), (1,)).item()]

# These variable includes:
        # self.input_tensor = input_tensor
        # self.model_action_name = model_action_name
        # self.model_bet_size = model_bet_size
        # self.baseline_action_name = baseline_action_name
        # self.baseline_bet_size = baseline_bet_size
        # self.action_label = action_label
        # self.bet_size_label = bet_size_label
        # self.baseline_inputs = baseline_inputs
        
# Baseline input is easy to view, the rest are just tensors

model_inputs = baseline_io_record.baseline_inputs
model_label_action_name = baseline_io_record.action_label
model_label_bet_size = baseline_io_record.bet_size_label*poker_dataset.bet_std + poker_dataset.bet_mean

model_call_size = model_inputs['call_size']*poker_dataset.call_std + poker_dataset.call_mean
model_pot_size = model_inputs['pot_size']*poker_dataset.pot_std + poker_dataset.pot_mean
model_stack_size = model_inputs['stack_size']*poker_dataset.stack_std + poker_dataset.stack_mean
model_min_raise = model_inputs['min_raise']*poker_dataset.min_raise_std + poker_dataset.min_raise_mean
model_hole_cards = model_inputs['hole_cards']
model_num_players = model_inputs['num_players']

baseline_model_inputs = model_io_record.baseline_inputs
baseline_label_action_name = model_io_record.action_label
baseline_label_bet_size = model_io_record.bet_size_label*poker_dataset.bet_std + poker_dataset.bet_mean

baseline_call_size = baseline_model_inputs['call_size']*poker_dataset.call_std + poker_dataset.call_mean 
baseline_pot_size = baseline_model_inputs['pot_size']*poker_dataset.pot_std + poker_dataset.pot_mean
baseline_stack_size = baseline_model_inputs['stack_size']*poker_dataset.stack_std + poker_dataset.stack_mean
baseline_min_raise = baseline_model_inputs['min_raise']*poker_dataset.min_raise_std + poker_dataset.min_raise_mean
baseline_hole_cards = baseline_model_inputs['hole_cards']
baseline_num_players = baseline_model_inputs['num_players']

print("Model Sample Call Size", model_call_size.item())
print("Model Sample Pot Size", model_pot_size.item())
print("Model Sample Stack Size", model_stack_size.item())
print("Model Sample Min Raise", model_min_raise.item())
print("Model Sample Hole Cards", model_hole_cards)
print("Model Sample Num Players", model_num_players.item())

print("\nModel Label Action Name", ACTION_NAMES[model_label_action_name.argmax().item()])
print("Model Label Bet Size", model_label_bet_size.item())

print("\nBaseline Sample Call Size", baseline_call_size.item())
print("Baseline Sample Pot Size", baseline_pot_size.item())
print("Baseline Sample Stack Size", baseline_stack_size.item())
print("Baseline Sample Min Raise", baseline_min_raise.item())
print("Baseline Sample Hole Cards", baseline_hole_cards)
print("Baseline Sample Num Players", baseline_num_players.item())

print("\nBaseline Label Action Name", ACTION_NAMES[baseline_label_action_name.argmax().item()])
print("Baseline Label Bet Size", baseline_label_bet_size.item())




Model Sample Call Size 3.878053903579712
Model Sample Pot Size 24.664764165878296
Model Sample Stack Size 4894.966045379639
Model Sample Min Raise 22.366286277770996
Model Sample Hole Cards [('9d',), ('Qs',)]
Model Sample Num Players 6

Model Label Action Name calls
Model Label Bet Size 0.25

Baseline Sample Call Size 3.878053903579712
Baseline Sample Pot Size 18.715341329574585
Baseline Sample Stack Size 2514.8030415344238
Baseline Sample Min Raise 22.366286277770996
Baseline Sample Hole Cards [('10c',), ('Js',)]
Baseline Sample Num Players 4

Baseline Label Action Name calls
Baseline Label Bet Size 0.25
